## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-29-17-02-24 +0000,nypost,Trump admin joins lawsuit against UCLA medical...,https://nypost.com/2026/01/29/us-news/trump-jo...
1,2026-01-29-17-02-00 +0000,wsj,The White House and Senate Democrats are movin...,https://www.wsj.com/politics/policy/white-hous...
2,2026-01-29-17-00-20 +0000,nyt,New Food Stamp Rules Are Confusing Grocery Sto...,https://www.nytimes.com/2026/01/29/business/sn...
3,2026-01-29-17-00-00 +0000,wsj,What CEOs Are—and Are Not—Saying About ICE and...,https://www.wsj.com/business/what-ceos-areand-...
4,2026-01-29-16-59-54 +0000,nyt,Democratic Governors Promise Accountability Af...,https://www.nytimes.com/2026/01/29/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2434/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,52
40,ice,28
41,minneapolis,22
442,minnesota,20
30,new,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
333,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...,137
250,2026-01-29-03-00-00 +0000,wsj,Fed Chair Jerome Powell’s decision on whether ...,https://www.wsj.com/economy/central-banking/je...,135
51,2026-01-29-15-47-21 +0000,nyt,Trump Moves to Sell Former Home of Capital’s T...,https://www.nytimes.com/2026/01/28/us/politics...,115
223,2026-01-29-05-47-03 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...,115
316,2026-01-28-23-10-05 +0000,bbc,Watch: How Alex Pretti shooting led Trump to s...,https://www.bbc.com/news/videos/cj6wgelw62do?a...,113


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
333,137,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...
147,102,2026-01-29-12-00-01 +0000,startribune,AI image of Alex Pretti’s killing is the lates...,https://www.startribune.com/ai-image-of-alex-p...
250,71,2026-01-29-03-00-00 +0000,wsj,Fed Chair Jerome Powell’s decision on whether ...,https://www.wsj.com/economy/central-banking/je...
1,71,2026-01-29-17-02-00 +0000,wsj,The White House and Senate Democrats are movin...,https://www.wsj.com/politics/policy/white-hous...
243,70,2026-01-29-03-15-00 +0000,wsj,Google co-founder Sergey Brin has contributed ...,https://www.wsj.com/politics/policy/sergey-bri...
115,62,2026-01-29-13-31-00 +0000,wsj,Democratic Sen. Amy Klobuchar of Minnesota ann...,https://www.wsj.com/politics/elections/sen-amy...
223,53,2026-01-29-05-47-03 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...
60,46,2026-01-29-15-30-20 +0000,nypost,NYC private school teacher sent revenge porn o...,https://nypost.com/2026/01/29/us-news/nyc-priv...
302,43,2026-01-28-23-56-00 +0000,wsj,FBI Agents Search Election Office in Georgia’s...,https://www.wsj.com/politics/elections/fbi-age...
71,39,2026-01-29-15-19-00 +0000,wsj,"After pandemic-era hiring binges, big companie...",https://www.wsj.com/economy/jobs/u-s-companies...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
